In [ ]:
import os
import torch
from torch import nn

from NeuroVisualizer.neuro_aux.AEmodel import UniformAutoencoder
from NeuroVisualizer.neuro_aux.utils import get_files

from helper.neuro_viz import get_dataloader_flat

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Neuro-Visualizer
This notebook creates the loss landscape from the NeuroVisualizer

In [ ]:
dataset_name = 'mnist'
model_name = 'CNN'
runs = [
#    "run-0011-CNN_mnist_32_0.9776",
#    "run-0012-CNN_mnist_32_0.9768"
#    "run-0007-CNN_mnist_128_0.9851",
    
    # With Residual
#    "run-0016-CNN_cifar10_128_0.8093", # Seed 42, SAM
#    "run-0018-CNN_cifar10_128_0.8499", # Seed 42
#    "run-0020-CNN_cifar10_128_0.8079", # Seed 11, SAM
#    "run-0022-CNN_cifar10_128_0.8519", # Seed 11
    
    # No Residual
    "run-0017-CNN_cifar10_128_0.8072", # Seed 42, SAM
    "run-0019-CNN_cifar10_128_0.8487", # Seed 42
    "run-0021-CNN_cifar10_128_0.8054", # Seed 11, SAM
    "run-0023-CNN_cifar10_128_0.8509", # Seed 11
]

In [ ]:
from helper.data_manager import load_training_data
results = []
run_ids = []
vis_id = ""

for run in runs:
    results.append(load_training_data(run))
    run_ids.append(results[-1]["ll_flattened_weights_dir"])

vis_id = ' x '.join(run_ids)
print(run_ids)
print(vis_id)

In [ ]:
model_file = f'ae_models/{vis_id}.pt'

In [ ]:
# Adjust this path to your folder
pt_files = []

for run_id in run_ids:
    model_folder = f"trainings/{run_id}"
    pt_files.append(get_files(model_folder, prefix="model-"))
    print(f"Found {len(pt_files[-1])} checkpoint files.")

pt_files_flat = [path for sublist in pt_files for path in sublist]

## Train AE Model
Run this part to train an AE-Model

In [ ]:
batch_size = 1 #4 - 32 Batch Size of AE Training

loader, normalizer = get_dataloader_flat(pt_files_flat, batch_size)

In [ ]:
torch.cuda.empty_cache()

Adjust: Choose the hidden dimension (that the model-GPU combination is still working with)

In [ ]:
input_dim = loader.dataset[0].shape[0]
print(f"Input dimension: {input_dim}")

latent_dim = 2
num_layers = 4

# Aggressive compression (scales with first hidden dim)
#h = [input_dim, 64, 32, 8]
h = [input_dim, 128, 64, 16]
ae = UniformAutoencoder(input_dim, num_layers, latent_dim, h=h).to(device)

#ae = UniformAutoencoder(input_dim, num_layers, latent_dim).to(device)

In [ ]:
total_params = sum(p.numel() for p in ae.parameters())
trainable_params = sum(p.numel() for p in ae.parameters() if p.requires_grad)

size_mb = total_params * 4 / (1024**2)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Approx. size: {size_mb:.2f} MB")

In [ ]:
from helper.neuro_viz import train_autoencoder

trained_model = train_autoencoder(
    model=ae,
    train_loader=loader,
    device=device,
    save_path=model_file,
    num_epochs=100,
    lr=0.009, # 0.001
    patience=15
)

## Visualize Trajectory

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from NeuroVisualizer.neuro_aux.AEmodel import UniformAutoencoder
from NeuroVisualizer.neuro_aux.utils import get_files, repopulate_model
from NeuroVisualizer.neuro_aux.trajectories_data import get_trajectory_dataloader

In [ ]:
batch_size = 4
loss_name = 'test_loss'
whichloss = 'mse' # this is CrossEntropyLoss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Get file list
# pt_files = get_files(model_folder, prefix="model-")

# Load AE
example_tensor = torch.load(pt_files[0][0], weights_only=True)
input_dim = example_tensor.shape[0]
latent_dim = 2
num_layers = 4
#h = [input_dim, 64, 32, 8]
h = [input_dim, 128, 64, 16]


ae_model = UniformAutoencoder(input_dim, num_layers, latent_dim, h=h).to(device)
ae_model.load_state_dict(torch.load(model_file, weights_only=True))
#ae_model.eval()

In [ ]:
# ---- Load data ----
from helper.neuro_viz import get_dataloader_flat

trajectory_loader, transform = get_dataloader_flat(pt_files_flat, batch_size, shuffle=False) #[:5] for Subset

### Repopulate original Model Architecture
**IMPORTANT: needs correct model**

In [ ]:
for result in results:
    print(result["model_info"])

In [ ]:
from helper.vision_classification import init_mlp_for_dataset, init_cnn_for_dataset
from helper.neuro_viz import Loss

#TODO Check the model:
model = init_cnn_for_dataset(dataset_name, conv_dims=[8, 16], kernel_sizes=[3, 3], hidden_dims=[32], dropout=0.25, residual=False).to(device)
#model = init_cnn_for_dataset(dataset_name, conv_dims=[8, 16], kernel_sizes=[3, 3], hidden_dims=[32], dropout=0.25, residual=True).to(device)

#model = init_cnn_for_dataset(dataset_name, conv_dims=[32, 64], kernel_sizes=[3, 3], hidden_dims=[128], dropout=0.25, residual=False).to(device)
#model = init_cnn_for_dataset(dataset, conv_dims=[32, 64], kernel_sizes=[3, 3], hidden_dims=[128], dropout=0.25, residual=True).to(device)

#model = init_cnn_for_dataset(dataset_name, conv_dims=[64, 128, 256], kernel_sizes=[5, 3, 3], hidden_dims=[256, 128], dropout=0.2, residual=True).to(device)
#model = init_mlp_for_dataset(dataset_name, hidden_dims=[254, 64], dropout=0.1).to(device)
loss_obj = Loss(dataset_name, device)

#### Compute trajectory (Coordinates and Loss)

In [ ]:
from helper.neuro_viz import compute_trajectory

trajectory_coordinates, trajectory_models, trajectory_losses = compute_trajectory(
    trajectory_loader,
    ae_model,
    transform,
    loss_obj,
    model,
    loss_name,
    whichloss,
    device,
)

In [ ]:
num_chunks = len(pt_files)
chunk_size = len(pt_files[0])

# Reshape the values
tr_losses = np.split(trajectory_losses.cpu().numpy(), num_chunks)  # list of arrays
tr_coordinates = np.split(trajectory_coordinates.cpu().numpy(), num_chunks)  # list of arrays

In [ ]:
import matplotlib.pyplot as plt

for i in range(num_chunks):
    plt.figure(figsize=(8, 4))

    plt.plot(results[i]["val_losses"], label='Logged Validation Loss', marker='o')
    plt.plot(reshaped[i], label='AE-Projected Validation Loss', marker='x')

    plt.legend()
    plt.title('Validation Loss: Training Log vs. AE-Projected Trajectory')
    plt.xlabel('Checkpoint Index')
    plt.ylabel('Loss (Cross Entropy)')
    plt.grid(True)
    plt.show()

In [ ]:
# Generate grid in latent space
from helper.neuro_viz import generate_latent_grid, compute_grid_losses
xx, yy, grid_coords = generate_latent_grid(
    min_map=-1, max_map=1,
    xnum=10, # 3 - 25
    device=device
)

# Decode grid and compute losses
#model = init_mlp_for_dataset(dataset_name, hidden_dims=[254, 64], dropout=0.1).to(device)

grid_losses = compute_grid_losses(
    grid_coords,
    transform,
    ae_model,
    model,
    loss_obj,
    loss_name,
    whichloss,
    device
)

# Reshape to grid
grid_losses = grid_losses.view(xx.shape)

In [ ]:
print(grid_losses.min().item(), grid_losses.max().item())

In [ ]:
rec_grid_models = ae_model.decoder(grid_coords)
rec_grid_models = rec_grid_models*transform.std.to(device) + transform.mean.to(device)

In [ ]:
from helper.neuro_viz import plot_loss_landscape

fig = plot_loss_landscape(
    xx, yy,
    grid_losses,
    tr_losses,
    tr_coordinates,
    rec_grid_models=rec_grid_models,
    draw_density=False,
    filled_contours=False
)

In [ ]:
# Save to PDF
os.makedirs('plots', exist_ok=True)
fig.savefig(f'plots/loss_landscape_{vis_id}.pdf', dpi=300, bbox_inches='tight', format='pdf')
print(f"Saved PDF to plots/loss_landscape_{vis_id}.pdf")

plt.show()